In [4]:
# 필수(?) 라이브러리 호출
import pandas as pd
import numpy as np

In [5]:
# 경고가 많아지면 곤란해서 추가
import warnings
warnings.filterwarnings('ignore')

In [6]:
# 일단 분석할 데이터를 가져오는데
df = pd.read_csv("../../raw_data/전체데이터_2011_코스피코스닥_v1.4.csv", thousands=',', encoding="CP949").reset_index(drop=True)
df

,회사명,거래소코드,회계년도,이자보상배율(이자비용)(IFRS),이자보상배율(이자비용)(IFRS연결)
0,(주)CMG제약,58820,11-Dec,3.94,3.53
1,(주)CMG제약,58820,12-Dec,-7.45,-7.75
2,(주)CMG제약,58820,13-Dec,0.73,0.70
3,(주)CMG제약,58820,14-Dec,3.63,3.60
4,(주)CMG제약,58820,15-Dec,-32.20,-32.22
...,...,...,...,...,...
15145,흥아해운(주),3280,15-Dec,0.79,1.02
15146,흥아해운(주),3280,16-Dec,0.04,0.21
15147,흥아해운(주),3280,17-Dec,-0.61,-0.42
15148,흥아해운(주),3280,18-Dec,-1.26,-1.16


In [7]:
# csv 한바퀴 돌립니다
for i in range(0,len(df)):

    # 개별 재무제표에 결측치가 없다면
    if not pd.isna(df.iloc[i]['이자보상배율(이자비용)(IFRS)']):
        # 연결 재무제표에도 결측치가 없다면
        if not pd.isna(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']):
            # 연결 재무제표 삽입
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']

        # 연결 재무제표가 결측치라면
        else:
            # 개별 재무제표 삽입
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS)']

    # 개별 재무제표에 결측치가 있다면
    else:
        # 연결 재무제표에 결측치가 없다면
        if not pd.isna(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']):
            # 연결 재무제표 삽입
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']

        # 둘 다 결측치가 존재하는 경우 통과
        else:
            pass

In [8]:
# 우리가 앞으로 작업을 엄청나게 복잡하게 할 걸 생각하면
# 중간중간 결과들을 데이터프레임으로 만들고 빼주는 게 좋습니다
df_2 = df[['회사명', '거래소코드', '회계년도', '이자보상배율']]
df_2

,회사명,거래소코드,회계년도,이자보상배율
0,(주)CMG제약,58820,11-Dec,3.53
1,(주)CMG제약,58820,12-Dec,-7.75
2,(주)CMG제약,58820,13-Dec,0.70
3,(주)CMG제약,58820,14-Dec,3.60
4,(주)CMG제약,58820,15-Dec,-32.22
...,...,...,...,...
15145,흥아해운(주),3280,15-Dec,1.02
15146,흥아해운(주),3280,16-Dec,0.21
15147,흥아해운(주),3280,17-Dec,-0.42
15148,흥아해운(주),3280,18-Dec,-1.16


In [9]:
# # 일단 우리의 소중한 결과물을 빼놓도록 하죠
# # 버전 올리는 거 잊지 마세요

# df_2.to_csv("../csv_data/이자보상배율_v1.0.csv", encoding="CP949")

In [10]:
df_2['회계년도'] = pd.to_datetime(df_2['회계년도'].str.upper(), format="%d-%b", yearfirst=False).dt.strftime("20%d")
#df_2['부실기업'] = 0

In [11]:
# 2년 연속일 때 컷
f_test = []
count = 0
company = df_2.iloc[0]['회사명']

for i in range(0, len(df_2)):

    # 회사명 다르면
    if company != df_2.iloc[i]['회사명']:
        # 회사명 기록하고
        company = df_2.iloc[i]['회사명']
        # 카운트 초기화하고
        count = 0

    # 2년 연속 걸릴 때
    if count == 2 :
        #df_2.loc[i, '부실기업'] = 1
        continue

    # 기록은 기록대로 다시 하고
    if df_2.iloc[i]['이자보상배율'] < 1:
        count += 1
    else:
        count = 0

    f_test.append(df_2.loc[i])
    #print(i, count)

In [12]:
# 전체 기록
f_test_x = []
count = 0
company = df_2.iloc[0]['회사명']

for i in range(0, len(df_2)):

    # 회사명 다르면
    if company != df_2.iloc[i]['회사명']:
        # 회사명 기록하고
        company = df_2.iloc[i]['회사명']
        # 카운트 초기화하고
        count = 0

    #df_2.loc[i, 'count'] = count

    # 이자보상배율이 0이면 탈출
    if df_2.iloc[i]['이자보상배율'] == 0:
        df_2.loc[i, '부실기업'] = 0
        f_test_x.append(df_2.loc[i])
        count = 0
        continue

    elif df_2.iloc[i]['이자보상배율'] < 1:
         count += 1

    else:
        count = 0

    # 이자보상배율에 따라 부실기업 여부 설정
    if count > 1 and df_2.iloc[i]['이자보상배율'] >= 1:
        df_2.loc[i, '부실기업'] = 0
        pass

    elif count > 1 and df_2.iloc[i]['이자보상배율'] < 1:
        df_2.loc[i, '부실기업'] = 1

    f_test_x.append(df_2.loc[i])
    #print(i, count)

In [13]:
# 리스트 저장하기
pd.DataFrame(f_test_x).to_csv("checking2.csv", encoding="CP949")

In [14]:
# 현재 데이터 기준으로 부실징후기업과 2년 연속 이.보.배 1 미만인 기업을 찾을 겁니다

f_company = [] # 부실징후기업을 저장하는 리스트
f_company_x = [] # 2년 연속 이자보상배율 1 미만인 기업 삽입
f_year = [] # 1년째 최초 연도
f_year_x = [] # 2년째 최초 연도
count = 0 # 부실징후기업 카운트용
company = 'x' # 부실징후기업 기록용

# 이거도 csv 한바퀴 돌립니다
for i in range(0, len(df_2)):

    # 회사명이 이미 기록된 상태에서 또 중복으로 적을 필요는 없습니다
    if company == df.iloc[i]['회사명']:
        # 다만 그 기업이 부실징후기업인데
        if count > 1:
            # 2년 연속이네요? 기록!!!
            if df_2.iloc[i]['이자보상배율'] < 1:
                f_company_x.append(company)
                f_year_x.append(df.iloc[i]['회계년도'])
                count = 0

                # 반복문에서 볼일 다 끝났으니 통과합니다
                continue

        # 그게 아니라면 반복문을 통과합니다
        continue

    # 이자보상배율이 1 미만인 경우 카운트를 늘립니다
    if df_2.iloc[i]['이자보상배율'] < 1:
            count += 1
    # 도중에 이자보상배율이 1 미만이면 탈출합니다
    else:
        count = 0

    # 한계 기업 클럽에 오셨습니다. 환영합니다.
    if count == 1:
        company = df_2.iloc[i]['회사명']
        f_company.append(company)
        f_year.append(df.iloc[i]['회계년도'])
        count += 1

In [15]:
# 분명히 이거보다 더 좋은 방법이 있을건데
# 맷돌이 굴러가질 않네요

# 이건 f_company 리스트에 들어잇는 데이터와 일치하면 True, 아니면 False를 반환합니다
df_4 = df_2.isin(f_company)
df_4

,회사명,거래소코드,회계년도,이자보상배율,부실기업
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,False,False,False
3,True,False,False,False,False
4,True,False,False,False,False
...,...,...,...,...,...
15145,True,False,False,False,False
15146,True,False,False,False,False
15147,True,False,False,False,False
15148,True,False,False,False,False


In [20]:
df_5 = [] # 임시 저장용 리스트

# 그렇게 만든 데이터프레임을 기존과 비교해서 True인 애들만 뽑아옵니다
for i in range(0, len(df_4)-1):
    if df_4.iloc[i]["회사명"] == True:
        df_5.append(df_2.loc[i])

# 그리고 저장을 합니다
df_5_frame = pd.DataFrame(df_5).reset_index(drop=True)
#df_5_frame.to_csv("00_한계기업후보_temp.csv", encoding="CP949")

In [17]:
# 2년 연속 이자 보상 배율이 1 미만인 기업들도 똑같이 해봅니다

df_6 = df_2.isin(f_company_x)
df_7 = []

for i in range(0, len(df_6)-1):
    if df_6.iloc[i]["회사명"] == True:
        df_7.append(df_2.loc[i])

df_7_frame = pd.DataFrame(df_7).reset_index(drop=True)

# # 중간중간 저장은 선택이 아닌 필수
# df_7_frame.to_csv("../csv_data/회생불가1차_v1.csv", encoding="CP949")

In [18]:
# 한계기업 후보 리스트가 어쨌든 메인이고
# 그 안에서 우리는 부실기업을 찾자는 것이고
df_5_frame['부실기업'] = 0
df_9 = df_5_frame.isin(df_7_frame)
for i in range(0, len(df_5_frame)):
    if df_9.iloc[i]['회사명'] == True:
        df_5_frame.loc[i, '부실기업'] = 1
df_5_frame

,회사명,거래소코드,회계년도,이자보상배율,부실기업
0,(주)CMG제약,58820,2011,3.53,1
1,(주)CMG제약,58820,2012,-7.75,1
2,(주)CMG제약,58820,2013,0.70,1
3,(주)CMG제약,58820,2014,3.60,1
4,(주)CMG제약,58820,2015,-32.22,1
...,...,...,...,...,...
9448,흥아해운(주),3280,2014,1.12,0
9449,흥아해운(주),3280,2015,1.02,0
9450,흥아해운(주),3280,2016,0.21,0
9451,흥아해운(주),3280,2017,-0.42,0


In [19]:
df_5_frame.to_csv("test2.csv", encoding="CP949")